##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Uso de DTensors com o Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/dtensor_keras_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/distribute/dtensor_keras_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Neste tutorial, você aprenderá a usar o DTensor com o Keras.

Por meio da integração do DTensor com o Keras, você pode reutilizar suas camadas e modelos atuais do Keras para criar e treinar modelos de aprendizado de máquina distribuídos.

Você treinará um modelo de classificação multicamada com os dados MNIST. Será demonstrado como definir o layout do modelo de subclasse, o modelo sequencial e o modelo funcional.

Para este tutorial, pressupõe-se que você já tenha lido o [guia de programação do DTensor](/guide/dtensor_overview) e conheça os conceitos básicos do DTensor, como `Mesh` e `Layout`.

Este tutorial é baseado em https://www.tensorflow.org/datasets/keras_example.

## Configuração

O DTensor faz parte da versão 2.9.0 do TensorFlow.

In [ ]:
!pip install --quiet --upgrade --pre tensorflow tensorflow-datasets

Em seguida, importe `tensorflow` e `tensorflow.experimental.dtensor`, e configure o TensorFlow para usar 8 CPUs virtuais.

Embora este exemplo use CPUs, o DTensor funciona da mesma forma em dispositivos com CPU, GPU ou TPU.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.experimental import dtensor

In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(
        phy_devices[0], 
        [tf.config.LogicalDeviceConfiguration()] * ncpu)
  
configure_virtual_cpus(8)
tf.config.list_logical_devices('CPU')

devices = [f'CPU:{i}' for i in range(8)]

## Geradores de números pseudoaleatórios determinísticos

É importante salientar que a API do DTensor requer que todos os clientes em execução tenham as mesmas sementes aleatórias para que o comportamento de inicialização dos pesos seja determinístico. Para conseguir isso, basta definir as sementes globais no Keras pela função `tf.keras.utils.set_random_seed()`.

In [ ]:
tf.keras.backend.experimental.enable_tf_random_generator()
tf.keras.utils.set_random_seed(1337)

## Criação de uma malha para paralelismo de dados

Este tutorial demonstra o treinamento Paralelo de Dados. A adaptação para os treinamentos Paralelo de Modelos e Paralelo Espacial pode ser tão simples quanto alterar para um conjunto diferente de objetos `Layout`. Confira o [tutorial aprofundado do DTensor sobre aprendizado de máquina](https://www.tensorflow.org/tutorials/distribute/dtensor_ml_tutorial) para ver mais informações sobre outros treinamentos distribuídos além do Paralelo de Dados.

O treinamento Paralelo de Dados é um esquema de treinamento paralelo usando com frequência, também utilizado, por exemplo, em `tf.distribute.MirroredStrategy`.

Com o DTensor, um loop de treinamento Paralelo de Dados usa uma `Mesh` que consiste de uma única dimensão de “lote”, em que cada dispositivo executa uma réplica do modelo, que recebe um fragmento do lote global.


In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=devices)

Como cada dispositivo executa uma réplica completa do modelo, as variáveis ​​do modelo devem ser totalmente replicadas na malha (sem fragmentação). Por exemplo, um Layout totalmente replicado para um peso de posto 2 nesta `Mesh` seria o seguinte:

In [ ]:
example_weight_layout = dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)  # or
example_weight_layout = dtensor.Layout.replicated(mesh, rank=2)

Um layout de um tensor de dados posto 2 nesta `Mesh` seria fragmentado na primeira dimensão (conhecida também como `batch_sharded`).

In [ ]:
example_data_layout = dtensor.Layout(['batch', dtensor.UNSHARDED], mesh)  # or
example_data_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)

## Criação de camadas do Keras com layout

No esquema Paralelo de Dados, costuma-se criar os pesos do modelo com um layout completamente replicado para que cada réplica do modelo possa fazer cálculos com os dados de entrada fragmentados.

Para configurar as informações do layout para seus pesos de camadas, o Keras expôs um parâmetro extra no construtor de camadas para a maioria das camadas integradas.

O seguinte exemplo cria um pequeno modelo de classificação de imagens com layout de pesos completamente replicado. Você pode especificar as informações de layout `kernel` e `bias` em `tf.keras.layers.Dense` por meio dos argumentos `kernel_layout` e `bias_layout`. A maioria das camadas integradas do Keras estão prontas para especificação explícita do `Layout` para os pesos das camadas.

In [ ]:
unsharded_layout_2d = dtensor.Layout.replicated(mesh, 2)
unsharded_layout_1d = dtensor.Layout.replicated(mesh, 1)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, 
                        activation='relu',
                        name='d1',
                        kernel_layout=unsharded_layout_2d, 
                        bias_layout=unsharded_layout_1d),
  tf.keras.layers.Dense(10,
                        name='d2',
                        kernel_layout=unsharded_layout_2d, 
                        bias_layout=unsharded_layout_1d)
])

Você pode verificar as informações do layout analisando a propriedade `layout` dos pesos.

In [ ]:
for weight in model.weights:
  print(f'Weight name: {weight.name} with layout: {weight.layout}')
  break

## Carregamento de um dataset e criação de um pipeline de entrada

Carregue um dataset MNIST e configure um pipeline de entrada de pré-processamento para ele. O dataset em si não está associado a qualquer informação de layout do DTensor. Há planos para melhorar a integração do DTensor para Keras com o `tf.data` em versões futuras do TensorFlow.


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [ ]:
batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Definição da lógica de treinamento do modelo

Agora defina a lógica de treinamento e avaliação do modelo.

A partir do TensorFlow 2.9, você precisa escrever um loop de treinamento personalizado para um modelo do Keras com DTensor. Isso é feito para adicionar informações de layout adequadas nos dados de entrada, o que não está integrado às funções padrão `tf.keras.Model.fit()` ou `tf.keras.Model.eval()` do Keras. Haverá mais suporte ao `tf.data` em uma versão futura. 

In [ ]:
@tf.function
def train_step(model, x, y, optimizer, metrics):
  with tf.GradientTape() as tape:
    logits = model(x, training=True)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))
    
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'loss': loss_per_sample}
  return results

In [ ]:
@tf.function
def eval_step(model, x, y, metrics):
  logits = model(x, training=False)
  loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'eval_loss': loss_per_sample}
  return results

In [ ]:
def pack_dtensor_inputs(images, labels, image_layout, label_layout):
  num_local_devices = image_layout.mesh.num_local_devices()
  images = tf.split(images, num_local_devices)
  labels = tf.split(labels, num_local_devices)
  images = dtensor.pack(images, image_layout)
  labels = dtensor.pack(labels, label_layout)
  return  images, labels

## Métricas e otimizadores

Ao usar a API do DTensor com `Metric` e `Optimizer` do Keras, você precisará fornecer as informações de malha extras para que qualquer variável de estado e tensor internos funcionem com as variáveis do modelo.

- Para um otimizador, o DTensor tem um novo namespace experimental, o `keras.dtensor.experimental.optimizers`, em que muitos Otimizadores existentes no Keras são expandidos para receberem um argumento de `mesh` adicional. Em versões futuras, isso poderá ser combinado com os otimizadores principais do Keras.

- Para as métricas, você pode especificar a `mesh` diretamente para o construtor como argumento para torná-la uma `Metric` compatível com o DTensor.

In [ ]:
optimizer = tf.keras.dtensor.experimental.optimizers.Adam(0.01, mesh=mesh)
metrics = {'accuracy': tf.keras.metrics.SparseCategoricalAccuracy(mesh=mesh)}
eval_metrics = {'eval_accuracy': tf.keras.metrics.SparseCategoricalAccuracy(mesh=mesh)}

## Treinamento do modelo

O exemplo abaixo fragmenta os dados do pipeline de entrada na dimensão do lote e treina o modelo, que tem pesos completamente replicados.

Com 3 épocas, o modelo deve atingir uma exatidão de cerca de 97%.

In [ ]:
num_epochs = 3

image_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=4)
label_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

for epoch in range(num_epochs):
  print("============================") 
  print("Epoch: ", epoch)
  for metric in metrics.values():
    metric.reset_state()
  step = 0
  results = {}
  pbar = tf.keras.utils.Progbar(target=None, stateful_metrics=[])
  for input in ds_train:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)

    results.update(train_step(model, images, labels, optimizer, metrics))
    for metric_name, metric in metrics.items():
      results[metric_name] = metric.result()

    pbar.update(step, values=results.items(), finalize=False)
    step += 1
  pbar.update(step, values=results.items(), finalize=True)

  for metric in eval_metrics.values():
    metric.reset_state()
  for input in ds_test:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)
    results.update(eval_step(model, images, labels, eval_metrics))

  for metric_name, metric in eval_metrics.items():
    results[metric_name] = metric.result()
  
  for metric_name, metric in results.items():
    print(f"{metric_name}: {metric.numpy()}")


## Especificação do Layout para o código existente do modelo

Às vezes, você terá modelos que funcionam bem para seu caso de uso. A especificação de informações de `Layout` em cada camada individual do modelo demanda muito trabalho, exigindo muitas alterações.

Para ajudar a converter facilmente seu modelo atual do Keras para que ele funcione com a API do DTensor, você pode usar a nova API `dtensor.LayoutMap`, que permite especificar o `Layout` globalmente.

Primeiro, você precisa criar uma instância de `LayoutMap`, que é um objeto tipo dicionário contendo todo o `Layout` que você gostaria de especificar para os pesos do seu modelo.

O `LayoutMap` precisa de uma instância de `Mesh` na inicialização, que pode ser usada para fornecer o `Layout` replicado padrão para qualquer peso que não tenha o Layout configurado. Caso você queira que todos os pesos do seu modelo sejam somente replicados completamente, pode fornecer um `LayoutMap` vazio, e a malha padrão será usada para criar o `Layout` replicado.

O `LayoutMap` usa uma string como chave e um `Layout` como valor. Existe uma diferença de comportamento entre um dicionário Python comum e esta classe. A chave da string será tratada como uma expressão regular ao recuperar o valor.

### Modelo com subclasses

Considere o seguinte modelo, definido usando-se a sintaxe de modelo de subclasse do Keras.

In [ ]:
class SubclassedModel(tf.keras.Model):

  def __init__(self, name=None):
    super().__init__(name=name)
    self.feature = tf.keras.layers.Dense(16)
    self.feature_2 = tf.keras.layers.Dense(24)
    self.dropout = tf.keras.layers.Dropout(0.1)

  def call(self, inputs, training=None):
    x = self.feature(inputs)
    x = self.dropout(x, training=training)
    return self.feature_2(x)

Há 4 pesos neste modelo, que são `kernel` e `bias` para duas camadas `Dense`. Cada um deles é mapeado com base no caminho do objeto:

- `model.feature.kernel`
- `model.feature.bias`
- `model.feature_2.kernel`
- `model.feature_2.bias`

Observação: para modelos com subclasse, o atributo name, em vez do atributo `.name` da camada, é usado como a chave para recuperar o Layout no mapeamento. Isso é consistente com a conversão seguida pelos checkpoints do `tf.Module`. Para modelos complexos com mais do que poucas camadas, você pode [inspecionar manualmente os checkpoints](https://www.tensorflow.org/guide/checkpoint#manually_inspecting_checkpoints) para ver os mapeamentos de atributos.

Agora, defina o seguinte `LayoutMap` e aplique-o ao modelo.

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

with layout_map.scope():
  subclassed_model = SubclassedModel()

Os pesos do modelo são criados na primeira chamada, então faça uma chamada ao modelo com uma entrada do DTensor e confirme que os pesos tenham os layouts esperados.

In [ ]:
dtensor_input = dtensor.copy_to_mesh(tf.zeros((16, 16)), layout=unsharded_layout_2d)
# Trigger the weights creation for subclass model
subclassed_model(dtensor_input)

print(subclassed_model.feature.kernel.layout)

Dessa forma, você pode mapear o `Layout` para seus modelos rapidamente, sem atualizar o código existente. 

### Modelos sequencial e funcional

Para os modelos sequencial e funcional do Keras, você também pode usar o `LayoutMap`.

Observação: para os modelos funcional e sequencial, os mapeamentos são um pouco diferentes. As camadas do modelo não têm um atributo público vinculado ao modelo (embora você possa acessá-las por `model.layers` como uma lista). Nesse caso, use o nome da string como a chave. O nome da string sempre será único em um modelo.

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

In [ ]:
with layout_map.scope():
  inputs = tf.keras.Input((16,), batch_size=16)
  x = tf.keras.layers.Dense(16, name='feature')(inputs)
  x = tf.keras.layers.Dropout(0.1)(x)
  output = tf.keras.layers.Dense(32, name='feature_2')(x)
  model = tf.keras.Model(inputs, output)

print(model.layers[1].kernel.layout)

In [ ]:
with layout_map.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(16, name='feature', input_shape=(16,)),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(32, name='feature_2')
  ])

print(model.layers[2].kernel.layout)